<a href="https://colab.research.google.com/github/bigearn23/Coursera_Capsone/blob/main/Copy_of_Capstone_Project_Where_to_open_restaurant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



#1.   Introduction/Business Problem


Problem: I am a consultant for entrepreneurs and my clients often want advice on locations to start their business.  In this project, I will be focusing on a client that wants to open up an Italian restaurant in New York City, but not sure the best location. They are not looking at any particular area within NYC, but they want to start their business where there is limited competition nearby, or they would like a location that may have competition, but that competition has a below average customer rating. 



#2.   Data
I will be utilizing Foursquare location data to understand where existing Italian restaurants are currently located as well as how highly rated those restaurants are. I am searching for a location that has a limited number of Italian restaurants in the neighborhood, or I would also accept a fair amount of Italian restaurants nearby, but all with less than average reviews. I will be using the Foursqaure API to determine location data and capture reviews, so Client secret and Client ID will be required.

I will also be utilizing NYC neighborhood location data that includes longitude and latitude for each neighborhood to be able to identify which neighborhoods will be best fit for my client to open their Italian restaurant.

I will first take the NYC data and put into a dataframe that will give me the Longitude/Latitude of each neighborhood. From there, I will utilize Foursquare data to understand where other Italian Restaurants are in comparison to the neighborhood's Longitude/Latitude to understand which neighborhoods have the least density of Italian Restaurants. I will also be utilizing Foursquare data to look at customer reviews to understand where the highest rated Italian restaurants are and where they are not, so I can help my client fill a gap where there are not any local Italian Restaurants with good ratings.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

/bin/bash: conda: command not found
Libraries imported.


In [2]:
#Download the data
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [ ]:
#explore the data
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
newyork_data

In [4]:
#download the dataset
json_file = "/content/drive/My Drive/data sources/Neighborhoods_2012_polygons.json"

with open(json_file, 'r') as j:
     chicago_data = json.loads(j.read())
#!wget -q -O 'chicago_data.json' /content/drive/My Drive/data sources/Neighborhoods_2012_polygons.json
print('Data downloaded!')

Data downloaded!


In [5]:
#define a variable and view the first row
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [6]:
#Transform the data to a panda dataframe
#define the column names
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [7]:
#loops thru the data and populate dataframe
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [12]:
#view the first rows
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
#Use geopy to get the lat and long of NY, NY
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [9]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [21]:
#ENTER FOURSQUARE ID AND SECRET
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 200 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5XESDCU1MHDAVBSS4L3Q15PDOAMAPN0Z4KGCN2TJ3IB5Q40Y
CLIENT_SECRET:3RAK2RF2CSZLRRDENACPF4ZNLANCAGBWIXVZ2IEAXNYYZMEB


In [22]:
#Explore the first neighborhood
neighborhoods.loc[0, 'Neighborhood']

'Wakefield'

In [23]:
#get the neighborhood's long and lat
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


In [26]:
#Get a list of the top 100 italian restaurants that are in Wakefield neighborhood and within 2000 meters
LIMIT = 200 # limit of number of venues returned by Foursquare API
search_query = 'Italian'
radius = 2000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    search_query,
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=5XESDCU1MHDAVBSS4L3Q15PDOAMAPN0Z4KGCN2TJ3IB5Q40Y&client_secret=3RAK2RF2CSZLRRDENACPF4ZNLANCAGBWIXVZ2IEAXNYYZMEB&v=20180605&ll=40.89470517661,-73.84720052054902&query=Italian&radius=2000&limit=200'

In [27]:
#send the Get request
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fa560c4b3c24f7b3810692b'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-511edb6de4b0d58346fd272d-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d110941735',
         'name': 'Italian Restaurant',
         'pluralName': 'Italian Restaurants',
         'primary': True,
         'shortName': 'Italian'}],
       'delivery': {'id': '277050',
        'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
          'prefix': 'https://fastly.4sqi.net/img/general/cap/',
          'sizes': [40, 50]},
         'name': 'seamless'},
        'url': 'https://www.seamless.com/menu/patrizias-of-woodlawn-4358-katonah-ave-bronx/277050?affiliate=1131&utm_so

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
#clean the data and put into dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Patrizia's Of Woodlawn,Italian Restaurant,40.900638,-73.867724
1,Rivers Edge,Italian Restaurant,40.901218,-73.861698
2,Lincoln Lounge,Italian Restaurant,40.910671,-73.843943
3,Casa Mia,Italian Restaurant,40.905293,-73.858027
4,Axum Italian Restaurant,Food,40.882721,-73.856712


In [30]:
#see how many venues were returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [31]:
#create a function to repeat the same process for all neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            search_query,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
#run the above function for every neighborhood
ny_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'])

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [33]:
#check the size of the dataframe
print(ny_venues.shape)
ny_venues.head(15)

(8321, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Patrizia's Of Woodlawn,40.900638,-73.867724,Italian Restaurant
1,Wakefield,40.894705,-73.847201,Rivers Edge,40.901218,-73.861698,Italian Restaurant
2,Wakefield,40.894705,-73.847201,Lincoln Lounge,40.910671,-73.843943,Italian Restaurant
3,Wakefield,40.894705,-73.847201,Casa Mia,40.905293,-73.858027,Italian Restaurant
4,Wakefield,40.894705,-73.847201,Axum Italian Restaurant,40.882721,-73.856712,Food
5,Co-op City,40.874294,-73.829939,Fratelli's,40.863019,-73.843607,Italian Restaurant
6,Co-op City,40.874294,-73.829939,Olive Garden,40.864483,-73.827894,Italian Restaurant
7,Co-op City,40.874294,-73.829939,Sbarro,40.863591,-73.826911,Italian Restaurant
8,Fieldston,40.895437,-73.905643,Beccofino,40.905467,-73.900423,Italian Restaurant
9,Fieldston,40.895437,-73.905643,Salvatore's of Soho,40.887633,-73.907178,Italian Restaurant


In [36]:
#Analyze Each Neighborhood

ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Neighborhood,American Restaurant,Bagel Shop,Bakery,Bar,Breakfast Spot,Burger Joint,Butcher,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Event Space,Fast Food Restaurant,Food,Food & Drink Shop,Food Truck,French Restaurant,Gastropub,Gourmet Shop,Greek Restaurant,Hot Dog Joint,Hotel,Ice Cream Shop,Italian Restaurant,Jazz Club,Lounge,New American Restaurant,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Steakhouse,Tapas Restaurant,Vegetarian / Vegan Restaurant,Wine Bar
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
ny_onehot.shape

(8321, 40)

In [38]:
#Group the rows by neighborhood and take the mean of the frequency of each venue category
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped

,Neighborhood,American Restaurant,Bagel Shop,Bakery,Bar,Breakfast Spot,Burger Joint,Butcher,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Event Space,Fast Food Restaurant,Food,Food & Drink Shop,Food Truck,French Restaurant,Gastropub,Gourmet Shop,Greek Restaurant,Hot Dog Joint,Hotel,Ice Cream Shop,Italian Restaurant,Jazz Club,Lounge,New American Restaurant,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Steakhouse,Tapas Restaurant,Vegetarian / Vegan Restaurant,Wine Bar
0,Allerton,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.600000,0.000000,0.00,0.000000,0.133333,0.000000,0.000000,0.066667,0.000000,0.00,0.000000,0.000000,0.000000
1,Annadale,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,Arden Heights,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
3,Arlington,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
4,Arrochar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
5,Astoria,0.019608,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.019608,0.000000,0.019608,0.039216,0.000000,0.039216,0.000000,0.00,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,0.000000,0.686275,0.000000,0.00,0.000000,0.058824,0.000000,0.000000,0.039216,0.019608,0.00,0.000000,0.000000,0.019608
6,Astoria Heights,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.032258,0.032258,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.774194,0.000000,0.00,0.000000,0.096774,0.000000,0.000000,0.032258,0.000000,0.00,0.000000,0.000000,0.000000
7,Auburndale,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.850000,0.000000,0.00,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
8,Bath Beach,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.848485,0.000000,0.00,0.000000,0.090909,0.000000,0.000000,0.060606,0.000000,0.00,0.000000,0.000000,0.000000
9,Battery Park City,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.890000,0.000000,0.00,0.000000,0.030000,0.000000,0.000000,0.020000,0.000000,0.00,0.000000,0.000000,0.000000


In [40]:
ny_grouped.shape

(293, 40)

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
#create a new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Italian Restaurant,Pizza Place,Bar,Sandwich Place,Deli / Bodega,Coffee Shop,Wine Bar,Fast Food Restaurant,Event Space,Donut Shop
1,Annadale,Italian Restaurant,Wine Bar,Food Truck,Food,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
2,Arden Heights,Italian Restaurant,Wine Bar,Food Truck,Food,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
3,Arlington,Italian Restaurant,Wine Bar,Food Truck,Food,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
4,Arrochar,Italian Restaurant,Wine Bar,Food Truck,Food,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega


In [46]:
#cluster neighborhoods

kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 4, 4, 4, 1, 1, 4, 1, 4], dtype=int32)

In [48]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Clusters Labels', kmeans.labels_)

ny_merged = neighborhoods

# merge ny_grouped with neighborhoods to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Clusters Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,1.0,1.0,Italian Restaurant,Food,Wine Bar,Food Truck,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
1,Bronx,Co-op City,40.874294,-73.829939,4.0,4.0,Italian Restaurant,Wine Bar,Food Truck,Food,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
2,Bronx,Eastchester,40.887556,-73.827806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bronx,Fieldston,40.895437,-73.905643,1.0,1.0,Italian Restaurant,Diner,Deli / Bodega,Pizza Place,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Dessert Shop
4,Bronx,Riverdale,40.890834,-73.912585,1.0,1.0,Italian Restaurant,Diner,Deli / Bodega,Pizza Place,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Dessert Shop


In [55]:
#Replace NaN with 0 in dataframe
ny_merged['Cluster Labels'] = ny_merged['Cluster Labels'].fillna(0)

In [56]:
display(ny_merged.dtypes)
ny_merged['Cluster Labels'] = ny_merged['Cluster Labels'].astype(int) 
display(ny_merged.dtypes)

Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Clusters Labels           float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Clusters Labels           float64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [57]:
ny_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Clusters Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,1.0,1,Italian Restaurant,Food,Wine Bar,Food Truck,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
1,Bronx,Co-op City,40.874294,-73.829939,4.0,4,Italian Restaurant,Wine Bar,Food Truck,Food,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
2,Bronx,Eastchester,40.887556,-73.827806,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bronx,Fieldston,40.895437,-73.905643,1.0,1,Italian Restaurant,Diner,Deli / Bodega,Pizza Place,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Dessert Shop
4,Bronx,Riverdale,40.890834,-73.912585,1.0,1,Italian Restaurant,Diner,Deli / Bodega,Pizza Place,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Dessert Shop


In [58]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [59]:
#Examine Cluster 0

ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Eastchester,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Kingsbridge,0,Italian Restaurant,Diner,Pizza Place,Deli / Bodega,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Dessert Shop
6,Marble Hill,0,Italian Restaurant,Pizza Place,Diner,Wine Bar,Deli / Bodega,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Dessert Shop
11,Pelham Parkway,0,Italian Restaurant,Pizza Place,Bar,Sandwich Place,Restaurant,Deli / Bodega,Coffee Shop,Wine Bar,Cocktail Bar,Event Space
20,Melrose,0,Italian Restaurant,Pizza Place,Café,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop
22,Port Morris,0,Italian Restaurant,Pizza Place,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
23,Longwood,0,Italian Restaurant,Pizza Place,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
24,Hunts Point,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Soundview,0,Italian Restaurant,Pizza Place,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
27,Clason Point,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
#Examine Cluster 1
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,1,Italian Restaurant,Food,Wine Bar,Food Truck,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
3,Fieldston,1,Italian Restaurant,Diner,Deli / Bodega,Pizza Place,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Dessert Shop
4,Riverdale,1,Italian Restaurant,Diner,Deli / Bodega,Pizza Place,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Dessert Shop
7,Woodlawn,1,Italian Restaurant,Diner,Wine Bar,Food Truck,Food,Fast Food Restaurant,Event Space,Donut Shop,Dessert Shop,Deli / Bodega
13,Bedford Park,1,Italian Restaurant,Pizza Place,Deli / Bodega,Bar,Dessert Shop,Café,Wine Bar,Fast Food Restaurant,Event Space,Donut Shop
14,University Heights,1,Italian Restaurant,Wine Bar,Pizza Place,Dessert Shop,Deli / Bodega,Café,Sandwich Place,Bar,Breakfast Spot,Burger Joint
16,Fordham,1,Italian Restaurant,Pizza Place,Deli / Bodega,Café,Bar,Sandwich Place,Dessert Shop,Fast Food Restaurant,Event Space,Donut Shop
17,East Tremont,1,Italian Restaurant,Deli / Bodega,Bar,Dessert Shop,Pizza Place,Café,Wine Bar,Fast Food Restaurant,Event Space,Donut Shop
18,West Farms,1,Italian Restaurant,Deli / Bodega,Bar,Dessert Shop,Pizza Place,Café,Wine Bar,Fast Food Restaurant,Event Space,Donut Shop
19,High Bridge,1,Italian Restaurant,Coffee Shop,Sandwich Place,Café,Wine Bar,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop


In [61]:
#Examine Cluster 2
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Norwood,2,Italian Restaurant,Pizza Place,Diner,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Dessert Shop,Deli / Bodega
9,Williamsbridge,2,Pizza Place,Italian Restaurant,Food,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
25,Morrisania,2,Pizza Place,Italian Restaurant,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
40,Castle Hill,2,Pizza Place,Italian Restaurant,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
45,Edenwald,2,Food,Pizza Place,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
71,Cypress Hills,2,Pizza Place,Italian Restaurant,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
78,Coney Island,2,Pizza Place,Italian Restaurant,Hot Dog Joint,Ice Cream Shop,Wine Bar,Coffee Shop,Event Space,Donut Shop,Diner,Dessert Shop
85,Sea Gate,2,Pizza Place,Wine Bar,Coffee Shop,Food,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
90,City Line,2,Italian Restaurant,Pizza Place,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
146,Woodhaven,2,Italian Restaurant,Pizza Place,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega


In [62]:
#Examine Cluster 3
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
171,Broad Channel,3,Ice Cream Shop,Wine Bar,Food Truck,Food,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
178,Rockaway Beach,3,Ice Cream Shop,Bar,Pizza Place,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop
302,Hammels,3,Ice Cream Shop,Wine Bar,Food Truck,Food,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega


In [63]:
#Examine Cluster 4
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,4,Italian Restaurant,Wine Bar,Food Truck,Food,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
10,Baychester,4,Italian Restaurant,Wine Bar,Food Truck,Food,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
12,City Island,4,Italian Restaurant,Wine Bar,Food Truck,Food,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
15,Morris Heights,4,Italian Restaurant,Sandwich Place,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
50,Gravesend,4,Italian Restaurant,Sandwich Place,Restaurant,Pizza Place,Wine Bar,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
52,Sheepshead Bay,4,Italian Restaurant,Restaurant,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
60,Brownsville,4,Italian Restaurant,Wine Bar,Food Truck,Food,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
73,Starrett City,4,Italian Restaurant,Wine Bar,Food Truck,Food,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
74,Canarsie,4,Italian Restaurant,Wine Bar,Food Truck,Food,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
77,Manhattan Beach,4,Italian Restaurant,Restaurant,Wine Bar,Coffee Shop,Fast Food Restaurant,Event Space,Donut Shop,Diner,Dessert Shop,Deli / Bodega
